## **Prompt Generation for FLUX**

In [1]:
from google import genai
from dotenv import load_dotenv
from IPython.display import display, Markdown
import os
import json

load_dotenv()

def get_brand_data(brand_id=None, brand_name=None):

    try:
        with open("brands.json", "r") as f:
            data = json.load(f)
            
        brands = data.get("all_brands", [])
        
        for brand in brands:
            # Check by ID if provided
            if brand_id is not None:
                if str(brand.get("brand_id")) == str(brand_id):
                    return brand
            
            # Check by Name if provided
            if brand_name is not None:
                if brand.get("brand_name", "").lower() == brand_name.lower():
                    return brand
                    
        return None
    except FileNotFoundError:
        print("Error: brands.json not found.")
        return None
    except Exception as e:
        print(f"Error reading brands.json: {e}")
        return None



In [ ]:
def llm_chatbot_for_flux(user_query, brand_id=None, brand_name=None):

    # 1. Retrieve Brand Data
    brand_data = get_brand_data(brand_id=brand_id, brand_name=brand_name)
    
    if not brand_data:
        return "Error: Brand not found."
    
    # 2. Serialize Brand Context for Prompt
    brand_context_str = json.dumps(brand_data, indent=2)
    
    # 3. Load Prompts
    try:
        with open("prompts/llm-gemini/user.txt", "r", encoding="utf-8") as f:
            user_prompt_template = f.read()
        
        with open("prompts/llm-gemini/system.txt", "r", encoding="utf-8") as f:
            system_prompt = f.read()
    except FileNotFoundError as e:
        return f"Error: Prompt file not found - {e}"

    # 4. Inject Variables into User Prompt
    user_prompt = user_prompt_template.replace("{{brand_context}}", brand_context_str)
    user_prompt = user_prompt.replace("{{user_query}}", str(user_query))

    # 5. Call Gemini API
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        return "Error: GEMINI_API_KEY not found in environment variables."

    try:
        from google.genai import types
        client = genai.Client(api_key=api_key)
        
        response = client.models.generate_content(
            model="gemini-2.5-flash-preview-09-2025",
            contents=user_prompt,
            config=types.GenerateContentConfig(
                system_instruction=system_prompt,
                response_mime_type="application/json"
            )
        )
        return response.text
    except Exception as e:
        return f"Error calling Gemini API: {e}"

query1 = "a lady in a modern attire looking at the mixer operating it for makinga juice along with talking on mobile phone ensuring that the mixer is silent and she is happily talking. the mixer should be placed on a table in a nice kitchen and the mixer's photo will be attached to flux, make sure no changes should be done on the product" 

### **Testing**

In [ ]:
# Test 1: Simple Request
if __name__ == "__main__":
    print('--------------------------------------------------')
    print("--- Test 1: Havells  - Simple Request ---")
    query1 = "create a Indian lady looking at the mixer using it along with talking on mobile phone ensure the mixer is silent and she is happily talking on her phone. the mixer should be placed on a table in a nice kitchen and mixer photo will be attached to flux" 
    print('query -', query1)
    response1 = llm_chatbot_for_flux(user_query=query1, brand_name="Havells")
    print("\n[LLM Response]:\n")
    display(Markdown(f"```json\n{response1}\n```"))
    #print(response1)
    print('--------------------------------------------------')


--------------------------------------------------
--- Test 1: Havells  - Simple Request ---
query - a lady in a modern attire looking at the mixer operating it for makinga juice along with talking on mobile phone ensuring that the mixer is silent and she is happily talking. the mixer should be placed on a table in a nice kitchen and the mixer's photo will be attached to flux, make sure no changes should be done on the product

[LLM Response]:



```json
{
  "scene": "A well-lit, clean kitchen countertop during a peaceful morning routine. The focus is on effortless, modern multitasking.",
  "subjects": [
    {
      "description": "A young adult woman in modern, smart casual attire (e.g., linen shirt, comfortable trousers).",
      "position": "Standing at a kitchen island or counter, facing the appliance.",
      "action": "Operating a compact countertop appliance (blender/mixer) to make a juice while simultaneously holding a smartphone to her ear and smiling, demonstrating comfortable conversation despite the appliance being active. She appears relaxed and focused on her conversation.",
      "appearance": "Happy, confident, and professional yet relaxed.",
      "colors": []
    }
  ],
  "environment": {
    "background": "A minimalist, contemporary kitchen interior featuring light stone countertops and sleek, handleless cabinetry.",
    "setting": "A modern, aspirational residential kitchen setting.",
    "time_of_day": "Bright morning."
  },
  "style": {
    "visual_style": "Premium lifestyle photography, sharp focus, clean lines, and contemporary realism. Emphasis on ease of use and modern living.",
    "mood": "Effortless, peaceful, reliable, and sophisticated.",
    "color_palette": [
      "#FFFFFF",
      "#E31E24"
    ]
  },
  "lighting": "Bright, soft natural daylight entering from a large window, creating subtle highlights on the surfaces.",
  "composition": "Medium shot, framed to show the woman's face and upper body, the appliance on the counter, and the hand holding the phone. Shallow depth of field to isolate the subject and appliance against the background.",
  "camera": {
    "angle": "Eye level or slightly elevated.",
    "lens": "50mm prime lens.",
    "depth_of_field": "Shallow, isolating the subject and appliance, emphasizing detail."
  },
  "typography": {
    "text": "",
    "style": "",
    "color": "",
    "placement": ""
  },
  "brand_placement": {
    "logo_area": "Bottom right corner.",
    "logo_treatment": "empty space reserved for brand logo placement"
  }
}
```

--------------------------------------------------


In [9]:
# Test 1: Simple Request
if __name__ == "__main__":
    print('--------------------------------------------------')
    print("--- Test 1: Starshine  ---")
    query1 = """
    A modern Indian family of four is seated around a stylish wooden dining table. father sitting on the center and right side mother and left side both kids in modern Indian apartment. blue sky afternoon atmosphere with soft natural light streaming through large windows and sheer curtains. Everyone is looking at the empty bowl of chutney, His expression shows mild surprise and realization. The young son and young daughter sit opposite each other, surprised, Plates filled with Indian food like paneer sabzi, rotis, and salad are neatly arranged. The empty green garlic chutney bowl is clearly visible in fathers hand. Modern home décor, neutral tones, cozy and premium lifestyle feel, realistic photography style, shallow depth of field, high detail, natural skin tones, authentic Indian family emotions, no text, no branding, cinematic framing, lifestyle food advertising aesthetic
    """
    print('query -', query1)
    response1 = llm_chatbot_for_flux(user_query=query1, brand_name="Starshine")
    print("\n[LLM Response]:\n")
    display(Markdown(f"```json\n{response1}\n```"))
    #print(response1)
    print('--------------------------------------------------')


--------------------------------------------------
--- Test 1: Starshine  ---
query - 
    A modern Indian family of four is seated around a stylish wooden dining table. father sitting on the center and right side mother and left side both kids in modern Indian apartment. blue sky afternoon atmosphere with soft natural light streaming through large windows and sheer curtains. Everyone is looking at the empty bowl of chutney, His expression shows mild surprise and realization. The young son and young daughter sit opposite each other, surprised, Plates filled with Indian food like paneer sabzi, rotis, and salad are neatly arranged. The empty green garlic chutney bowl is clearly visible in fathers hand. Modern home décor, neutral tones, cozy and premium lifestyle feel, realistic photography style, shallow depth of field, high detail, natural skin tones, authentic Indian family emotions, no text, no branding, cinematic framing, lifestyle food advertising aesthetic
    

[LLM Response]:



```json
{
  "scene": "A candid moment of surprise captured at a stylish dining table in a modern, light-filled apartment during a family lunch.",
  "subjects": [
    {
      "description": "An adult man, representing the father, holding a small empty serving bowl in his hand.",
      "position": "Seated center at the head of the table",
      "action": "Showing an expression of mild, humorous realization and surprise while looking at the empty bowl.",
      "appearance": "Modern casual Indian attire, clean and contemporary style.",
      "colors": [
        "#FFFFFF",
        "#000000",
        "#0078FF"
      ]
    },
    {
      "description": "An adult woman, representing the mother.",
      "position": "Seated to the right of the man at the table",
      "action": "Reacting with a slightly surprised expression, looking towards the empty bowl.",
      "appearance": "Modern Indian attire, relaxed and stylish.",
      "colors": [
        "#FFFFFF",
        "#000000"
      ]
    },
    {
      "description": "A young boy, representing the son.",
      "position": "Seated across the table from the mother",
      "action": "Looking up with an expression of confusion and surprise.",
      "appearance": "Casual, youthful clothing.",
      "colors": [
        "#0078FF",
        "#FFFFFF"
      ]
    },
    {
      "description": "A young girl, representing the daughter.",
      "position": "Seated across the table from the father and next to the young boy",
      "action": "Showing a wide-eyed, slightly amused look of surprise.",
      "appearance": "Casual, bright clothing.",
      "colors": [
        "#FFFFFF",
        "#0078FF"
      ]
    }
  ],
  "environment": {
    "background": "Large window filtering bright afternoon light through sheer white curtains; subtle blue sky visible outside.",
    "setting": "Modern Indian apartment dining room with contemporary wooden dining table and chairs. Table is set with authentic Indian cuisine (paneer, rotis, salad) and clean, simple tableware. Cozy and premium décor in neutral tones.",
    "time_of_day": "Mid-afternoon"
  },
  "style": {
    "visual_style": "Realistic lifestyle photography, high-definition, cinematic framing, contemporary advertising aesthetic.",
    "mood": "Engaging, authentic, surprising, humorous, comfortable.",
    "color_palette": [
      "#FFFFFF",
      "#000000",
      "#0078FF"
    ]
  },
  "lighting": "Soft, diffused natural light pouring through sheer curtains (window light), high contrast details.",
  "composition": "Medium close-up shot focusing on the family gathered around the dining table, emphasizing facial expressions and the central empty bowl.",
  "camera": {
    "angle": "Eye level",
    "lens": "Shallow depth of field, high-quality prime lens look (e.g., 85mm)",
    "depth_of_field": "Shallow, focusing sharply on the father and the bowl, blurring the background slightly"
  },
  "typography": {
    "text": "",
    "style": "",
    "color": "",
    "placement": ""
  },
  "brand_placement": {
    "logo_area": "Bottom right corner of the frame",
    "logo_treatment": "empty space reserved for brand logo placement"
  }
}
```

--------------------------------------------------


--------------------------------------------------
--- Test 3: IKEA - Complex Request ---


create a new year post


[LLM Response]:



```json
{
  "scene": "A well-lit, functional home interior designed for planning and reflection.",
  "subjects": [
    {
      "description": "A person of ambiguous gender, dressed in simple, comfortable clothing, sitting at a small, light-colored table.",
      "position": "Centered, slightly facing the viewer.",
      "action": "Gently writing or journaling in an open notebook, focused on setting intentions for the new year.",
      "appearance": "Comfortable and approachable, suggesting a moment of peace.",
      "colors": [
        "#FFFFFF",
        "#000000"
      ]
    }
  ],
  "environment": {
    "background": "A clean, white wall with integrated modular shelving units holding simple items like potted plants, books, and storage boxes.",
    "setting": "A minimalist, organized home office nook or reading space, emphasizing utility and comfort.",
    "time_of_day": "Mid-morning, bright and clear."
  },
  "style": {
    "visual_style": "Clean, high-key Scandinavian-inspired interior photography. Focus on functional design and simple geometry.",
    "mood": "Optimistic, inspiring, fresh, and calm, symbolizing new beginnings and organization.",
    "color_palette": [
      "#FFFFFF",
      "#FFCC00",
      "#0058A3",
      "#000000"
    ]
  },
  "lighting": "Soft, diffused natural light pouring into the room, emphasizing bright whites and clean lines.",
  "composition": "Rule of thirds composition, focusing on the subject's interaction with the functional furniture setup.",
  "camera": {
    "angle": "Slightly elevated eye-level.",
    "lens": "Standard lens (50mm equivalent).",
    "depth_of_field": "Shallow depth of field, keeping the subject and immediate workspace sharp while softly blurring the background storage elements."
  },
  "typography": {
    "text": "",
    "style": "",
    "color": "",
    "placement": ""
  },
  "brand_placement": {
    "logo_area": "Bottom right corner of the frame.",
    "logo_treatment": "Empty space reserved for brand logo placement"
  }
}
```

--------------------------------------------------


In [ ]:
# Test 1: Simple Request
if __name__ == "__main__":
    print('--------------------------------------------------')
    print("--- Test 1: Under Armour - Simple Request ---")
    query1 = "create a new year post"
    print('query -', query1)
    response1 = llm_chatbot_for_flux(user_query=query1, brand_name="Under Armour")
    print("\n[LLM Response]:\n")
    display(Markdown(response1))
    #print(response1)
    print('--------------------------------------------------')


--------------------------------------------------
--- Test 1: Under Armour - Simple Request ---
query - create a new year post

[LLM Response]:

--------------------------------------------------


In [28]:
# Test 2: Specific Visual Request
if __name__ == "__main__":
    print('--------------------------------------------------')
    print("--- Test 2: Nike - Specific Visual Request ---")
    query2 = "create a new year post with a caption new year new will and logo on top right corner use color red black and white with a nice athletic in the background"
    display(Markdown(query2))
    response2 = llm_chatbot_for_flux(user_query=query2, brand_id=2)
    print("\n[LLM Response]:\n")
    display(Markdown(f"```json\n{response2}\n```"))
    #print(response2)
    print('--------------------------------------------------')


--------------------------------------------------
--- Test 2: Nike - Specific Visual Request ---


create a new year post with a caption new year new will and logo on top right corner use color red black and white with a nice athletic in the background


[LLM Response]:



```json
{
  "project_title": "New Year Athletic Motivation Post",
  "aspect_ratio": "4:5",
  "prompt_description": "A high-contrast, dynamic photograph capturing a highly determined athletic figure in intense motion (e.g., mid-sprint or high-intensity training). The visual style is gritty and cinematic, emphasizing sweat, speed, and raw effort. The background is dark and abstract, perhaps a heavily shadowed urban or gym setting. The lighting uses strong directional sources to create deep shadows and highlights muscular definition. The overall mood is intense, resilient, and forward-looking.",
  "style": {
    "mood": "Intense, determined, empowering, fresh start",
    "visual_style": "High-contrast athletic photography, modern grit, dynamic movement, minimalistic graphic elements",
    "color_palette": [
      "#000000",
      "#FFFFFF",
      "#E01D24"
    ]
  },
  "elements": {
    "subject": {
      "type": "Professional Athlete",
      "details": "Focused expression, muscular physique, visible effort and motion, wearing performance athletic gear in black/dark tones."
    },
    "background": {
      "setting": "Abstract or shadowed training environment (track, dark gym, urban structure)",
      "lighting": "Dramatic, contrast-heavy studio lighting with strong shadows"
    },
    "graphic_overlays": {
      "details": "Minimal use of vibrant red (#E01D24) for graphic accent lines or highlights to emphasize movement and urgency."
    }
  },
  "text_overlay": {
    "headline": "THE YEAR OF GRIT",
    "placement": "Center-aligned, large scale, potentially partially obscured by the athlete's movement.",
    "font_style_description": "Extra bold, geometric, blocky sans-serif, reflecting strength and modern performance.",
    "color": "#FFFFFF"
  },
  "brand_placement": {
    "logo_area": "Empty space reserved for brand logo placement, positioned in the top right corner of the composition.",
    "logo_treatment": "Monochromatic white outline or solid red fill."
  }
}
```

--------------------------------------------------


In [20]:
if __name__ == "__main__":
    # Test 3: Complex Request with Comparison
    print('--------------------------------------------------')    
    print("--- Test 3: Under Armour - Complex Request ---")
    query3 = "create a new year post also suggest caption for it logo on top left corner use colors red black and white use image provided to get a better understanding of how other poster are made for it"
    display(Markdown(query3))
    response3 = llm_chatbot_for_flux(user_query=query3, brand_name="Under Armour")
    print("\n[LLM Response]:\n")
    display(Markdown(f"```json\n{response3}\n```"))
    #print(response3)
    print('--------------------------------------------------')


--------------------------------------------------
--- Test 3: Under Armour - Complex Request ---


create a new year post also suggest caption for it logo on top left corner use colors red black and white use image provided to get a better understanding of how other poster are made for it


[LLM Response]:



```json
{
  "scene": "A powerful and dynamic image capturing an athlete pushing their limits at the start of the New Year, focused on relentless determination and grit.",
  "subjects": [
    {
      "description": "Muscular athlete (gender neutral) captured mid-explosion during a vertical leap or explosive sprint start, wearing black performance athletic gear that shows sweat and motion. Their expression is intensely focused and determined.",
      "position": "Center frame, occupying the majority of the foreground.",
      "action": "Jumping, moving upwards and towards the viewer with maximum velocity.",
      "colors": [
        "#000000",
        "#FF0000"
      ],
      "color_match": "exact"
    },
    {
      "description": "Under Armour brand logo (UA icon)",
      "position": "Top left corner",
      "action": "Static branding element",
      "colors": [
        "#FFFFFF"
      ],
      "color_match": "exact"
    }
  ],
  "style": "Cinematic, high-contrast sports photography with motion blur emphasizing speed and power. Gritty texture and hyper-realistic rendering of sweat and muscle definition.",
  "color_palette": [
    "#000000",
    "#FFFFFF",
    "#FF0000"
  ],
  "lighting": "Dramatic, harsh directional lighting (spotlights) from above and the side, creating deep, intense shadows and highlighting the athlete's outline and muscle tone.",
  "mood": "Intense, driven, empowering, and highly ambitious.",
  "background": "Dark, industrial gym setting (concrete floor and exposed metal beams) slightly out of focus, drenched in shadows to isolate the subject.",
  "composition": "Low angle shot with dynamic diagonal lines leading the eye upward, emphasizing the athlete's power and height. Use of negative space in the upper corners.",
  "camera": {
    "angle": "Extreme low angle (worm's eye view)",
    "lens": "Fast prime lens (e.g., 50mm or 85mm) to emphasize speed and depth.",
    "depth_of_field": "Very shallow, focusing sharply only on the athlete's face and immediate action point."
  },
  "typography": {
    "text": "YEAR ONE: RELOAD.",
    "style": "Bold, condensed, uppercase sans-serif font, resembling performance gear text.",
    "color": "#FF0000",
    "placement": "Bottom center, large scale, contrasting sharply with the dark background."
  }
}
```

--------------------------------------------------


## **Prompt Generation for Nano Banana**

In [ ]:
from google import genai
from dotenv import load_dotenv
import os
import json

load_dotenv()

def get_brand_data(brand_id=None, brand_name=None):

    try:
        with open("brands.json", "r") as f:
            data = json.load(f)
            
        brands = data.get("all_brands", [])
        
        for brand in brands:
            # Check by ID if provided
            if brand_id is not None:
                if str(brand.get("brand_id")) == str(brand_id):
                    return brand
            
            # Check by Name if provided
            if brand_name is not None:
                if brand.get("brand_name", "").lower() == brand_name.lower():
                    return brand
                    
        return None
    except FileNotFoundError:
        print("Error: brands.json not found.")
        return None
    except Exception as e:
        print(f"Error reading brands.json: {e}")
        return None



In [ ]:
def llm_chatbot_for_nano_banana(user_query, brand_id=None, brand_name=None):

    # 1. Retrieve Brand Data
    brand_data = get_brand_data(brand_id=brand_id, brand_name=brand_name)
    
    if not brand_data:
        return "Error: Brand not found."
    
    # 2. Serialize Brand Context for Prompt
    brand_context_str = json.dumps(brand_data, indent=2)
    
    # 3. Load Prompts
    try:
        with open("prompts/llm-gemini/user.txt", "r") as f:
            user_prompt_template = f.read()
        
        with open("prompts/llm-gemini/system.txt", "r") as f:
            system_prompt = f.read()
    except FileNotFoundError as e:
        return f"Error: Prompt file not found - {e}"

    # 4. Inject Variables into User Prompt
    user_prompt = user_prompt_template.replace("{{brand_context}}", brand_context_str)
    user_prompt = user_prompt.replace("{{user_query}}", str(user_query))

    # 5. Call Gemini API
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        return "Error: GEMINI_API_KEY not found in environment variables."

    try:
        from google.genai import types
        client = genai.Client(api_key=api_key)
        
        response = client.models.generate_content(
            model="gemini-2.5-flash-preview-09-2025",
            contents=user_prompt,
            config=types.GenerateContentConfig(
                system_instruction=system_prompt,
                response_mime_type="application/json"
            )
        )
        return response.text
    except Exception as e:
        return f"Error calling Gemini API: {e}"
